In [9]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.layers import MaxPooling2D

In [ ]:
# COMP3710 Project
# Question 4: Segment the ISICs data set with the Improved UNet [1] with all labels having a minimum Dice similarity
# coefficient of 0.8 on the test set. [Normal Difficulty]

# Student Name: Xiao Sun
# Studeng Number: 45642586
